# Hyperparameter Tuning Coding Challenge

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, **YOUR SURNAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

## Overview

Within this coding challenge, we strengthen our ability to produce appropriate classification solutions by extending a base model with tuned hyperparameters. 

<br></br>

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/wine.jpg"
     alt="Some fine wine for your fine model"
     style="float: center; padding-bottom=0.5em"
     width=600px/>
Some fine wine for your fine modeling process. 
Photo by <a href="https://unsplash.com/@hermez777?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText"> Hermes Rivera</a> on Unsplash
</div>

The structure of this notebook is as follows:

 - First, we'll load our data to get a view of the predictor and response variables we will be modeling. 
 - We'll then preprocess our data, binarising the target variable and splitting up the data intro train and test sets. 
 - We then model our data using a Support Vector Classifier.
 - Following this modeling, we define a custom metric as the log-loss in order to evaluate our produced model.
 - Using this metric, we then take several steps to improve our base model's performance by optimising the hyperparameters of the SVC through a grid search strategy. 
 
### Imports 

In [18]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

## The Dataset 

For this coding challenge we'll be using the [Wine Quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Repository. 

This dataset consists of the following variables: 

 - fixed acidity
 - volatile acidity
 - citric acid
 - residual sugar
 - chlorides
 - free sulfur dioxide
 - total sulfur dioxide
 - density
 - pH
 - sulphates
 - alcohol
 - quality (score between 0 and 10)

### Reading in the data


**Note** the feature we will be predicting is quality, i.e. the label is 'quality' using classification.

In [63]:
df = pd.read_csv('https://raw.githubusercontent.com/Explore-AI/Public-Data/master/Data/classification_sprint/winequality.csv')
df.head(10)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,0,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,0,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,0,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,0,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,0,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,0,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


## Question 1 - Data Preprocessing

We would like to classify the wine according to it's quality using binary classification.
Write a function to preprocess the data so we can run it through the classifier. The function should:

* Convert the quality for lower quality wines (quality less than or equal to 5) to 0
* Convert the quality for higher quality wines (quality greater than or equal to 6) to 1
* Split the data into 75% training and 25% testing data
* Set random_state to equal 42 for this internal method. 

_**Function Specifications:**_
* Should take a dataframe
* Standardise the features using sklearn's ```StandardScaler```
* Convert the quality labels into a binary labels
* Should fill nan values with zeros
* Should return two `tuples` of the form `(X_train, y_train), (X_test, y_test)`.

In [75]:
### START FUNCTION
def data_preprocess(df):
    from sklearn.preprocessing import StandardScaler
    df = df.replace(np.nan, 0)
    df.loc[df['quality'] <= 5, 'quality'] = 0
    df.loc[df['quality'] >= 6, 'quality'] = 1
    sc = StandardScaler()
    X = df.iloc[:, :-1].values
    X = sc.fit_transform(X)
    y = df['quality'].values
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

    return (x_train, y_train), (x_test, y_test)

### END FUNCTION

In [80]:
(X_train, y_train), (X_test, y_test) = data_preprocess(df)

In [ ]:
print(X_train[:2])

_**Expected Outputs:**_
```python
(X_train, X_test,y_train, y_test)=data_splitting(df)
print(X_train[:2])
print(y_train[:2])
print(X_test[:2])
print(y_test[:2])


[[-0.57136659  0.07127869 -0.48054096  1.17914161 -0.09303318 -0.79974133
   0.0830898  -0.15472329 -0.36573452  0.13010447  0.06101473  0.25842195]
 [-0.57136659  1.50396711 -0.72301571  0.56008035 -0.63948302 -0.05776881
  -0.70572997  0.62379657  0.16787589 -0.86828773 -0.47467813 -0.99931317]]

[1 0]

[[-0.57136659 -0.15493527 -0.54115965  0.90400327 -0.66050032 -0.31460545
   0.53384396  0.03990667 -1.35291379 -0.26925241 -0.34075491  1.18076103]
 [-0.57136659  0.29749266 -1.20796522  2.8987562  -0.80762143 -0.45729248
  -0.19863155 -0.22549783 -1.03274754 -0.7185289  -0.87644778  0.25842195]]

[1 1]  
``` 

## Question 2 - Model Training

Now that you have processed your data, let's jump straight into model fitting. Write a function that should:
* Instantiate a `SVC` model.
* Train the `SVC` model with default parameters.
* Return the trained SVC model. 

_**Function Specifications:**_
* Should take two numpy `arrays` as input in the form `(X_train, y_train)`.
* Should return an sklearn `SVC` model which has a random state of 40 and gamma set to 'auto'.
* The returned model should be fitted to the data.

In [81]:
### START FUNCTION
def train_SVC_model(X_train,y_train):
    # your code here
    clasif = SVC(random_state=40, gamma='auto')
    clasif.fit(X_train, y_train)
    return clasif

### END FUNCTION

In [82]:
svc = train_SVC_model(X_train,y_train)
svc.classes_

array([0, 1], dtype=int64)

In [ ]:
svc = train_SVC_model(X_train,y_train)
svc.classes_


_**Expected Outputs:**_

```python
svc.classes_
```
```
array([0, 1], dtype=int64)
```

## Question 3 - Model Testing

Now that you've trained your model. It's time to test its accuracy, however, we'll be using a custom scoring function for this. Create a function that implements the log loss function:

$$\Large  H(p,q)= - \frac{1}{N}\sum_{i=1}^{N} -ylog(\hat{y}_{i}) - (1- y)log(1 - \hat{y}_{i})$$

_**Function Specifications:**_
* Should take two numpy `arrays` as input in the form `y_true` and `y_predicted`.
* Should return a `float64` for the log loss value rounded to 7 decimal places.

_**Hint:**_ the numpy subtract function can be used to perform a calculation across an array of values

In [204]:
### START FUNCTION
def custom_scoring_function(y_true, y_pred):
    # your code here
    epsilon = 1e-15
    y_pred = np.maximum(epsilon, y_pred)
    y_pred = np.minimum(1-epsilon, y_pred)
    ll = np.sum((-y_true*np.log(y_pred)-np.subtract(1,y_true)*np.log(np.subtract(1,y_pred))))
    ll = ll * ((-1/len(y_pred))) *-1
    return np.round(ll, 7)

### END FUNCTION

In [205]:
y_pred = svc.predict(X_test)
print('Log Loss value: ', custom_scoring_function(y_test, y_pred))
print('Accuracy: ',round(accuracy_score(y_test,y_pred),4))

Log Loss value:  7.3329468
Accuracy:  0.7877


In [196]:
y_pred = svc.predict(X_test)
print('Log Loss value: ', custom_scoring_function(y_test, y_pred))
print('Accuracy: ',round(accuracy_score(y_test,y_pred),4))

Log Loss value:  -7.3329468
Accuracy:  0.7877


_**Expected Outputs:**_
```python
print('Log Loss value: ',custom_scoring_function(y_test,y_pred))
print('Accuracy: ',accuracy_score(y_test,y_pred))
```

> ```
Log Loss value:  1.2540518
Accuracy:  0.9637
```

## Hyperparameter Optimization

### Question 4.1 - Getting model parameters
In order to improve the accuracy of our classifier, we have to search for the best possible model (`SVC` in this case) parameters. However, we first have to find out what parameters can be tuned for the given model. Write a function that returns a list of available hyperparameters for a given model. 

_**Function Specifications:**_
* Should take in an sklearn model (estimator) object.
* Should return a list of parameters for the given model.

In [139]:
### START FUNCTION
def get_model_hyperparams(model):
    # your code here
    param_list = dict(model.get_params())
    params = list(param_list.keys())
    return params

### END FUNCTION

In [206]:
get_model_hyperparams()

TypeError: get_params() missing 1 required positional argument: 'self'

_**Expected Outputs:**_

```python
get_model_hyperparams(SVC)
```

> ```
['C',
 'cache_size',
 'class_weight',
 'coef0',
 .
 .
 .
 'shrinking',
 'tol',
 'verbose']
```

### Question 4.2 - Hyperparameter Search
The next step is define a set of `SVC` hyperparameters to search over. Write a function that searches for optimal parameters using the given dictionary of hyperparameters:

- C_list = [0.1, 1, 10]
- {C: 0.1, 1, 10}
- gamma_list = [0.01, 0.1, 1]
- {gamma: 0.01, 0.1, 1}
- D = {'C':[0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}

and using `custom_scoring_function` from **Question 3** above as a custom scoring function (_**Hint**_: Have a look at at the `make_scorer` object in sklearn `metrics`).

_**Function Specifications:**_
* Should define a parameter grid using the given list of `SVC` hyperparameters
* Should return an sklearn `GridSearchCV` object with a cross validation of 5.

In [197]:
### START FUNCTION
def tune_SVC_model(X_train, y_train):
    # your code here
    from sklearn.metrics import make_scorer
    
    my_scorer = make_scorer(custom_scoring_function, greater_is_better=False)
    search = GridSearchCV(svc,
                   scoring=my_scorer,
                   cv=5,
                   param_grid={"C": [0.1, 1, 10],
                               "gamma": [0.01, 0.1, 1]})
    search.fit(X_train, y_train)
    best_clf = search.best_estimator_
    return search

### END FUNCTION

In [198]:
svc_tuned = tune_SVC_model(X_train, y_train)
y_pred = svc_tuned.predict(X_test)
print('Log Loss value: ',custom_scoring_function(y_test,y_pred))
print('Accuracy: ',round(accuracy_score(y_test,y_pred),4))

Log Loss value:  -11.8603575
Accuracy:  0.6566


In [ ]:
y_pred = svc_tuned.predict(X_test)
print('Log Loss value: ',custom_scoring_function(y_test,y_pred))
print('Accuracy: ',round(accuracy_score(y_test,y_pred),4))

_**Expected Outputs:**_
```python
print('Log Loss value: ',custom_scoring_function(y_test,y_pred))
print('Accuracy: ',accuracy_score(y_test,y_pred))
```

> ```
Log Loss value:  1.2115421
Accuracy:  0.9649
```

### Question 4.3 - Optimal model parameters
Write a function that returns the best hyperperameters for a given model (i.e. the `GridSearchCV`). 

_**Function Specifications:**_
* Should take in an sklearn GridSearchCV object.
* Should return a dictionary of optimal parameters for the given model.

In [200]:
### START FUNCTION
# function that returns best params
def get_best_params(model):
    # your code here
    results = dict(model.best_params_)
    return results
### END FUNCTION

In [201]:
get_best_params(svc_tuned)

{'C': 0.1, 'gamma': 1}

In [ ]:
get_best_params(svc_tuned)

_**Expected Outputs:**_
```python
get_best_params(svc_tuned)
```

> ```
{'C': 1, 'gamma': 1}
```